### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


### Resolución
----

#### Imports
----

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import random

#### Carga de datos
----

In [2]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

#### Vectorización
----

In [3]:
tfidfvect = TfidfVectorizer()

In [4]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

In [5]:
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [6]:
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

----
### Parte 1
----

In [11]:
random.seed(10)

In [ ]:
#Busco numeros randoms para los 5 documentos
max_doc = len(newsgroups_train.data)
docs_indxs = [random.randint(1, max_doc) for i in range(5)]

In [32]:
def doc_study(doc_idx, X_train, y_train, newsgroups_train):
    doc = newsgroups_train.data[doc_idx]
    cos_sim = cosine_similarity(X_train[doc_idx], X_train)[0]
    #Ordeno y obtengo los 5 más similares
    five_most_sim = np.argsort(cos_sim)[::-1][1:6]
    doc_class = newsgroups_train.target_names[y_train[doc_idx]]
    print(f"La clase del documento es: {doc_class}")
    print(f"La clase de los documentos más similares son: ")
    for i in five_most_sim:
        print(newsgroups_train.target_names[y_train[i]])
    print(f"Contenido del documento: ")
    print("**********INICIO DOCUMENTO**********")
    print(doc)
    print("***********FIN DOCUMENTO************")

    print(f"Contenido de los 5 documentos más similares")
    for i in five_most_sim:
        print()
        print("**********INICIO DOCUMENTO**********")
        print(newsgroups_train.data[i])
        print("***********FIN DOCUMENTO************")
        print()



In [33]:
#Documento 1
doc_study(docs_indxs[0], X_train, y_train, newsgroups_train)

La clase del documento es: soc.religion.christian
La clase de los documentos más similares son: 
sci.med
talk.politics.misc
soc.religion.christian
soc.religion.christian
soc.religion.christian
Contenido del documento: 
**********INICIO DOCUMENTO**********
Does anyone know any good decipleship trainning program during min August 
to end of Sept.  Or any missionary programs.
I currently belong to the Missionary Alliance Church in Oregon.
Please reply by mail.

thanks.
***********FIN DOCUMENTO************
Contenido de los 5 documentos más similares

**********INICIO DOCUMENTO**********


does anyone know?

-- 
***********FIN DOCUMENTO************


**********INICIO DOCUMENTO**********
   

does anyone have the e-mail address for the white house. if so please send it to
me thanks a lot.


    

***********FIN DOCUMENTO************


**********INICIO DOCUMENTO**********




    Hmmm... people in the americas before the time of Christ, children who
die young, etc. ?




   But of course, the

Observando los resultados anteriroes se puede apreciar lo siguiente: 

- La clase del documento base, coincide con 3 de los 5 documentos más similares
- La principla concidencia entre los dos docuemtnos con clase distitna es que el formato de teexto es muy similar. En los tres casos, parece tratarse de alguien consultadno algo en un foro, mail o clasificado. 

Teniendo esos dos puntos en cuenta, se podria decir que para este primer documento las similaridades encontrdas son coherentes

In [34]:
#Documento 2
doc_study(docs_indxs[1], X_train, y_train, newsgroups_train)

La clase del documento es: rec.autos
La clase de los documentos más similares son: 
sci.space
rec.autos
rec.autos
soc.religion.christian
talk.politics.guns
Contenido del documento: 
**********INICIO DOCUMENTO**********

OK...

5) How about a fuel gauge that *really* told you how much fuel was
   left.  Like, "can I make it to where the gas is $1.14 or should
   I get gouged right here at $1.35?"  Accurate to the tenth of a
   gallon would be great.
***********FIN DOCUMENTO************
Contenido de los 5 documentos más similares

**********INICIO DOCUMENTO**********


The gravity maneuvering that was used was to exploit 'fuzzy regions'.  These
are described by the inventor as exploiting the second-order perturbations in a
three body system.  The probe was launched into this region for the
earth-moon-sun system, where the perturbations affected it in such a way as to
allow it to go into lunar orbit without large expenditures of fuel to slow
down. The idea is that 'natural objects sometim

Observando los resultados anteriores se encuentran 2 clases iguales frente a 3 distintas. 
Si bien hay lo que parece ser una doirerencia grande en las clases de los documentos, los 4 más similares parece tratarse de docuemtnso de clasificados, mientras que el ultimo caso parece ser una transcriopcion de una entrevista.

En sintesis, los documentos presentan cierta similaridad, al menos en el tip'o de docuemtno en si y no tanto en la tematica.

In [36]:
#Documento 3
doc_study(docs_indxs[2], X_train, y_train, newsgroups_train)

La clase del documento es: comp.windows.x
La clase de los documentos más similares son: 
comp.os.ms-windows.misc
comp.graphics
comp.windows.x
comp.windows.x
comp.sys.mac.hardware
Contenido del documento: 
**********INICIO DOCUMENTO**********
We have recently obtained a copy of color_xterm (from export.lcs.mit.edu) after
seeing it mentioned in a previous article. On compilation, it reports the
following undefined symbols:
	_get_wmShellWidgetClass
	_get_applicationShellWidgetClass

	but still runs. When sending escape sequences to set the colour,
any colour comes out as black text on a black background. Any Ideas?

Has anyone used this program in colour successfully?

Can anyone point me to any other colour terminal emulators?

We are running OpenWindows 3 on Sun Sparcs running SunOS 4.1.3.

				advaTHANKSnce  Ben Kelley.
***********FIN DOCUMENTO************
Contenido de los 5 documentos más similares

**********INICIO DOCUMENTO**********
Hi.  I am having some interesting problems with m

En este casso, si bien la clase de los documentos no es exacaemten la misma en todos los casos, todos parten de una base similar, la cual podriamos clasificar como "Computancion", teniendo eso en cuenta por un lado todas las clases parecen ser similares.
Si tomas en cuenta el contedio de los documentos, los mismo tambioen parecen similares,,todo indica que el documetno orignal parece tratarse de una consutla respecto a un programa o libreria en un foro. Los docuemntos más similares encontrdos, refieren a docuemtnos similares, en todos los casos son consultas o respuestas en foros (en un caso parece mostrarse la discucion completa) donde el eje central paraece girar a un tipo de softwre o extension de colores, similar a la consutla del docuemtno base. 

Es decir, se enceutnra gran similaridad entre el docuemtno base, con los 5 más simlares

In [37]:
#Documento 4
doc_study(docs_indxs[3], X_train, y_train, newsgroups_train)

La clase del documento es: rec.autos
La clase de los documentos más similares son: 
rec.autos
rec.autos
rec.autos
soc.religion.christian
rec.autos
Contenido del documento: 
**********INICIO DOCUMENTO**********
I was wondering if anyone out in net-land have any opinions on MGs
in general.  I know they are not the most reliable cars around but
summer is approaching and they are convertibles `8^).  I'm interested
in a 75 MG but any opinions on MGs would be appreciated.  Thanks.
***********FIN DOCUMENTO************
Contenido de los 5 documentos más similares

**********INICIO DOCUMENTO**********
I was wondering if someone could point me to somewhere I could
find a list (and hopefully comparison) of all the convertibles that are
out these days. Seems like they are making a big comeback - but I'm not
sure where to look

***********FIN DOCUMENTO************


**********INICIO DOCUMENTO**********
HELP!!!
my wife has informed me that she wants a convertible for her next car.
We live in South Fl

Aqui se pueden observar que los documentos similares todos menos uno tiene una clase similar al docuemnto base. Admeás todos los docuemtnos refieren a consultas en foros o sisitos de consultas de seguimiento de autos como en el documento base.
El documento 4, es el que menos se parece, simbien es una cosnutla en foro, la tematica es muy amplica ya que es una consulta para saber que se hace en cierto grupo, la similaridad con este docuemto no parece ser muy buena. 

Redondeandoa, se enceutnra 4 docuemtbos con muy buena similaridad, mientras que se eneuecntra un docuemtno cuya similairdad es un tanto cuestionable. 

In [38]:
#Documento 5
doc_study(docs_indxs[4], X_train, y_train, newsgroups_train)

La clase del documento es: alt.atheism
La clase de los documentos más similares son: 
alt.atheism
alt.atheism
soc.religion.christian
alt.atheism
alt.atheism
Contenido del documento: 
**********INICIO DOCUMENTO**********
 
 
I see, it is not rational, but it is intellectual. Does madness qualify
as intellectual engagement, too?
 
 
 
Exactly why science uses mathematics. QM representation in natural language
is not supposed to replace the elaborate representation in mathematical
terminology. Nor is it supposed to be the truth, as opposed to the
representation of gods or religions in ordinary language. Admittedly,
not  every religion says so, but a fancy side effect of their inept
representations are the eternal hassles between religions.
 
And QM allows for making experiments that will lead to results that will
be agreed upon as being similar. Show me something similar in religion.
 
 
 
With the effect that the models presented are useless. And one can argue
that the other way around, 

Todos los documentos tratan temas de contexto relaciodad a la religion y oprinciapelmtne el ateismo. La princiapl mdiferncia esta en la longitd del documeto base con respecto a los documentos similares. Fuera de ello, todos parecen tratrse de articulos referidos al ateismo. En general resuoltan tenener una buena similaridade con respeco al base. 